In [ ]:
! pip install langchain langchain-core --quiet

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # Lee el archivo .env

True

#### Sensibilidad de un prompt

In [ ]:
# Comprobamos cuan sensible es un LLM al prompt
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_groq import ChatGroq

gemma1 = ChatGroq(model="gemma-7b-it",
                   temperature=0,
                   max_tokens=50)
messages = [
    SystemMessage(content="Eres un esquimal de Groenlandia."),
    HumanMessage(content="""Te gusta el helado? Debes responder si o no. 
        """),             # Agregar unicamente
]
print(gemma1.invoke(messages).content)

#### Plantilla PromptTemplate
Propósito del ejemplo  <br>
Mostrar cómo utilizar la plantilla PromptTemplate empleando PromptTemplate como constructor directo.


##### Variables dinámicas y el método cadena f de Python

In [ ]:
# Empleando el metodo cadena f
variable1 = "Buenos dias"
variable2 = "Gustavo"
print(f"{variable1}, como esta usted? Me llamo {variable2}")

##### Empleando el constructor PromptTemplate

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    template="{saludo}, cómo está usted? Me llamo {nombre}",
    input_variables=["saludo", "nombre"],  )

prompt=prompt_template.format(saludo="Buenos dias", nombre="Gustavo")
print(prompt)
print(type(prompt))
print(type(prompt_template))

##### Parámetro template fuera de la plantilla

In [ ]:
from langchain_groq import ChatGroq
from langchain.document_loaders import WebBaseLoader

loader =WebBaseLoader(web_path="https://www.paradigmadigital.com/dev/soluciones-streaming-kafka")
doc2 = loader.load()[0]

llm = ChatGroq(model="mixtral-8x7b-32768",
                   temperature=1,
                   max_tokens=200)

template_resumen = \
"""
Quiero un resumen del texto que se encuentra entre triples comillas simples.
El resumen no debe superar  las {num_palabras} palabras.
Muestra las ideas principales en formato de bullets.
Como maximo usa {num_bullets} bullet points.
'''{input}'''
"""
# Construir la plantilla
prompt_template = PromptTemplate(template=template_resumen, 
                input_variables=["input','num_palabras","num_bullets"])

# Asignar valores
messages = prompt_template.format(input=doc2.page_content, 
                                  num_palabras=300, num_bullets=3)
# Invocar el modelo
res = llm.invoke(input=messages)
print(res.content)

##### Empleando el método from_template
Propósito del ejemplo  <br>
Mostrar cómo utilizar la plantilla PromptTemplate empleando el método from_template.


In [ ]:
prompt_1 = PromptTemplate.from_template("Estoy por viajar a {ciudad}. ")
intercala = " Por favor, "
prompt_2 = PromptTemplate.from_template("dígame qué hacer en {tiempo_de_estadia}.")

prompt = (prompt_1 + intercala + prompt_2)
prompt.format(ciudad="Singapur", tiempo_de_estadia="un mes")

#### Modular el comportamiento del modelo
Propósito del ejemplo <br>
Introducir el uso de un template más estructurado que los vistos anteriormente y mostrar cómo el prompt puede condicionar el comportamiento de un LLM. 


In [ ]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

template = """ 
Responde la consulta de acuerdo con el contexto que se da en Contexto.
###
Contexto: Eres un genetista que estudia la evolución de las especies y estás \
particularmente interesado en los cambios anatómicos con el tiempo.
###
Pregunta: "Cuantos dedos tiene un {especie1}, {especie2}, y {especie3}?"
###
Respuesta: 
"""

prompt_template = PromptTemplate.from_template(template=template)    
prompt = prompt_template.format(especie1="elefante", especie2="lagarto", especie3="ser humano") # lagarto, ser humano, elefante

llm = ChatGroq(model="llama3-8b-8192",
               temperature=0,
               max_tokens=300)

print(llm.invoke(prompt).content)
print("**********************************************")
#print(prompt)

##### Guardar un prompt en el disco duro
Propósito del ejemplo <br>
Mostrar cómo guardar una plantilla de prompt y recuperarla para otro uso. Se almacena en formato JSON.

In [ ]:
prompt_template.save("./directorio/mi_prompt.json") # Guardar en archivo JSON

In [ ]:
# Y recuperarlo con
from langchain_core.prompts import load_prompt

recupera_prompt = load_prompt("./directorio/mi_prompt.json")

assert prompt_template == recupera_prompt
print(recupera_prompt)

#### Ejemplo 1: Resumen de texto
Propósito del ejemplo <br>
Implementar un código que resume un texto ajustándose a instrucciones.


In [ ]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

template = """
%INSTRUCCIONES:
Por favor devuelva un resumen del texto adjunto empleando menos de 100 palabras.

%TEXT:
{text}
"""
texto = open("directorio/ConstitucionArgentina.txt", encoding='utf-8').read()

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)
prompt_final = prompt.format(text=texto)
#print(prompt_final)

llm = ChatGroq(model="llama3-8b-8192",
              temperature=0.25,
              max_tokens=150)

salida = llm.invoke(prompt_final)
print(salida.content)

##### Plantilla FewShotPromptTemplate - Ejemplo 1
Propósito del ejemplo  <br>
Implementar una plantilla FewShotPromptTemplate. Documentar el uso de PromptTemplate y los atributos prefix y suffix.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

mis_ejemplos = [
    {"pregunta": "La capital de Perú es:", "respuesta": "Lima"},
    {"pregunta": "El autor de El Quijote es:", "respuesta": "Cervantes"},
]
template_modelo = """
Usuario: {pregunta}
Asistente: {respuesta} """

# Se crea la plantilla definiendo las variables dinámicas y asignando example_format a la plantilla utilizando PromptTemplate
example_prompt = PromptTemplate(
    template=template_modelo,  #"Pregunta: {pregunta}\n{respuesta}"
    input_variables=["pregunta", "respuesta"],
)
print(example_prompt.format(**mis_ejemplos[0]))

##### Sigue la implementación de la plantilla FewShotPromptTemplate

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate

prefix=("Sigue el estilo de respuesta que se muestra en mis_ejemplos. "
            "La respuesta debe limitarse a unas pocas palabras.\n"
            "Preferentemente solo el nombre de la localidad o de la persona.")
# Se instancia un objeto de la clase `FewShotPromptTemplate`  
few_shot_prompt = FewShotPromptTemplate(
    # El prefix es una instrucción para que el LLM sepa que debe hacer con estos ejemplos
    prefix=prefix,    
    
    # le pasamos los ejemplos
    examples=mis_ejemplos,
    
    # le pasamos la plantilla
    example_prompt=example_prompt,

    # El suffix es donde se agrega el input del usuario para generar el texto que sigue
    suffix= "Responde esta pregunta: {input}" ,
    
    # indicamos la variable que se agregará al template
    input_variables=["input"],
)

Finalmente se concluye la construcción del prompt con format, que asigna valor a input.

In [ ]:
# Generamos el prompt con la función format()
entrada = few_shot_prompt.format(input="El automovilista argentino mas famoso es:")
print(entrada)

In [ ]:
llm.invoke(entrada).content

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch 

##### FewShotPromptTemplate - Ejemplo 2
Propósito del ejemplo <br>
Mostrar cómo con ejemplos se orienta al modelo para que distinga el significado de una palabra en base al contexto. Se trata de lograr que asiento, del verbo asentir, no se confunda con silla o banco.

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain_groq import ChatGroq

# Inicializar el modelo
llm = ChatGroq(model="llama3-8b-8192")

In [ ]:
ejemplos = [{"input":"asiento", "output":"afirmo"},
           {"input":"asiento", "output":"consiento"},
           {"input":"asiento", "output":"digo que si"},]
ej_prompt = PromptTemplate.from_template(
    "Palabra:{input}\nExplicacion:{output}")

In [ ]:
prompt = FewShotPromptTemplate(
    examples=ejemplos,
    example_prompt=ej_prompt,
    prefix="Responda en español.",
    suffix="Palabra:{input}",
    input_variables=["input"],
)
llm.invoke(prompt.format(input="asiento")).content

#### Plantilla ChatPromptTemplate
Propósito del ejemplo <br>
El ejemplo emplea la clase ChatPromptTemplate y el método from_messages para incorporar la lista de mensajes a chat_template.


In [13]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un experto guía de turismo internacional."),
        ("human", "Buen día. Planifico un viaje y necesito asesoramiento."),
        ("ai", "Por supuesto que le ayudaré! A dónde viaja?"),
        ("human", "{destino}"),
        ("ai", "Buena opción! Tengo varias recomendaciones para {destino}."),
        ("human", "Genial cuénteme.")
    ]
)


##### Se construye el prompt

In [15]:
# Con format_messages devuelve una lista
messages2 = chat_template.format_messages(destino="Paris")
print("format_messages devuelve tipo: ", type(messages2))
print("*******************************************************")
print(messages2[2])   

format_messages devuelve tipo:  <class 'list'>
*******************************************************
content='Por supuesto que le ayudaré! A dónde viaja?'


In [17]:
# Con format_prompt devuelve un PromptValue
messages1 = chat_template.format_prompt(destino="Paris") 
print("format_prompt devuelve tipo: ", type(messages1))
print("*******************************************************")
print(messages1.to_string())   # metodos to_string y to_messages
print("*******************************************************")

format_prompt devuelve tipo:  <class 'langchain_core.prompt_values.ChatPromptValue'>
*******************************************************
System: Eres un experto guía de turismo internacional.
Human: Buen día. Planifico un viaje y necesito asesoramiento.
AI: Por supuesto que le ayudaré! A dónde viaja?
Human: Paris
AI: Buena opción! Tengo varias recomendaciones para Paris.
Human: Genial cuénteme.
*******************************************************


In [18]:
# Se pasa el prompt al modelo
from langchain_groq import ChatGroq

chat = ChatGroq(model="llama3-8b-8192")
res1 = chat.invoke(messages1)  
print("Con format_prompt: ", res1.content)
print("**************************************")
res2 = chat.invoke(messages2)
print("Con format_messages: ", res2.content )

Con format_prompt:  Paris, la ciudad de la luz, es una de las más románticas y fascinantes del mundo. Hay tantas cosas que ver y hacer allí que no sé por dónde empezar.

 Primero, hay que mencionar los lugares emblemáticos como la Torre Eiffel, el Arco de Triunfo, la Catedral de Notre-Dame y el Museo Louvre, donde se encuentra la Mona Lisa. Pero también hay muchos lugares más ocultos y emocionantes, como el Jardín de Luxemburgo, el Panteón o el Mercado de Puerta de San Jacques.

Además, Paris es conocida por sus calles empedradas y sus cafés, como el Café de Flore o el Les Deux Magots, donde puede disfrutar de un café o un desayuno como los parisienses.

Y no podemos olvidar la moda y el arte, con diseñadores como Chanel, Dior y Yves Saint Laurent, y museos como el Musée Rodin o el Musée d'Orsay, que albergan obras de artistas como Monet, Renoir y Van Gogh.

También hay muchas opciones para disfrutar de la noche parisiense, como el Barrio Latino, el Quartier Latin o el Marais, donde pu

##### Variable única

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un experto guía de turismo internacional."),
        ("human", "Buen día. Planifico un viaje y necesito asesoramiento."),
        ("ai", "Por supuesto que le ayudaré! A dónde viaja?"),
        ("human", "Viajo a {destino}"),
        ("ai", "Buena opción! Tengo varias recomendaciones para {destino}."),
        ("human","Gracias!")
    ]
)
prompt = chat_template.invoke("Roma")
prompt

##### Variables múltiples

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un experto guía de turismo internacional."),
        ("human", "Buen día. Planifico un viaje y necesito asesoramiento."),
        ("ai", "Por supuesto que le ayudaré! A dónde viaja?"),
        ("human", "Viajo a dos ciudades {destino1} y {destino2}"),
        ("ai", "Buena opción! Tengo varias recomendaciones tanto para {destino1} como para {destino2}."),
        ("human","Gracias!")
    ]
)
prompt = chat_template.invoke({"destino1":"Roma", "destino2":"Paris"})
prompt

##### Empleando las clases SystemMessage, HumanMessage y AIMessage
Propósito del ejemplo <br>
Las respuestas previas muestran que internamente los roles human, system y ai en las tuplas han sido remplazados por las clases HumanMessage, SystemMessage y AIMessage, respectivamente, cada una con un parámetro content. Esto sugiere que se pueden emplear las clases correspondientes para construir los mensajes, como se ve en este ejemplo.


In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="Eres un experto guía de turismo internacional."),
        HumanMessage(content= "Buen día. Planifico un viaje y necesito asesoramiento."),
        AIMessage(content= "Por supuesto que le ayudaré! A dónde viaja?"),
        HumanMessage(content=  "Deseo viajar a Paris"),
        AIMessage(content="Buena opción! Tengo varias recomendaciones para ese destino."),
        HumanMessage(content= "Gracias!")
    ]
)
prompt_final = chat_template.format_messages()
print()
print("**** Sigue la respuesta del modelo. ****")
from langchain_groq import ChatGroq

modelo_chat = ChatGroq(model="llama3-8b-8192",
               temperature=0.25,
               max_tokens=150)
res = modelo_chat.invoke(prompt_final)
print(res.content)


**** Sigue la respuesta del modelo. ****
De hecho, París es una ciudad con una gran cantidad de lugares históricos, culturales y de entretenimiento. ¿Qué tipo de experiencia está buscando? ¿Le gustaría visitar lugares emblemáticos como la Torre Eiffel, el Louvre o Notre-Dame? ¿O prefiere explorar barrios como Montmartre, Le Marais o Saint-Germain-des-Prés? ¿Tiene algún interés en particular, como la moda, el arte o la gastronomía?

También puedo recomendarte algunos consejos prácticos para planificar su viaje, como la mejor época para visitar, cómo llegar al aeropuerto, dónde aloj


#### Plantilla MessagePromptTemplate
Propósito del ejemplo  <br>
Implementar una plantilla MessagePromptTemplate combinando las clases HumanMessage y HumanMessagePromptTemplate.


In [1]:
from langchain.prompts.chat import HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate

In [5]:
texto = "Setenta balcones y ninguna flor."
template = ChatPromptTemplate.from_messages(
[
    SystemMessagePromptTemplate.from_template(
     "Eres un experto traductor multilingüe; por favor traduce del {lenguaje_original} al {traducir_al}."),
    HumanMessage(content="Ayudame a traducir este texto."),
    AIMessage(content="Por favor provea el texto"),   
    HumanMessagePromptTemplate.from_template(texto)
   ])
# Se aplica el método format_messages de ChatPromptTemplate. Devuelve mensajes.
prompt_final = template.format_messages(lenguaje_original="español", traducir_al="francés", texto=texto)
# Se pasa el prompt al modelo
from langchain_groq import ChatGroq

chat = ChatGroq(model="llama3-8b-8192",
               temperature=0.25,
               max_tokens=150)   # Se instancia un objeto ChatGroq
res = chat.invoke(prompt_final)  # Se pasa el prompt al modelo
print(res.content)

Soixante-dix balcons et pas une fleur.

(Note: "Setenta" is translated to "soixante-dix" in French, as "setenta" is the Spanish word for "seventy", and "balcones" is translated to "balcons". "Ni" is translated to "pas", which is a more common way to express "no" or "none" in French. Finally, "flor" is translated to "fleur".)


#### Chain-of-Thought 1

In [4]:
from langchain_groq import ChatGroq

template = """
Tengo una cinta de 120 cm de largo. Debo cortarla en trozos de 15 cm de largo cada uno. 
Cuantos trozos de 15 cm de largo puedo obtener?
Piense paso a paso.
"""
llm = ChatGroq(model="llama3-8b-8192",
               temperature=0.1,
               max_tokens=1000,
               verbose=True)   # Se instancia un objeto ChatGroq

prompt = PromptTemplate(template=template, input_variables=[])
prompt_final = prompt.format()


# Procesar con un LLM
response = llm.invoke(prompt_final)
print(response.content)

¡Vamos a resolver esto paso a paso!

1. Primero, necesitamos encontrar cuántos centímetros de cinta tenemos en total: 120 cm.
2. Luego, necesitamos encontrar cuántos trozos de 15 cm de largo podemos cortar de la cinta. Para hacer esto, podemos dividir el total de centímetros de cinta (120 cm) entre la longitud de cada trozo (15 cm).
3. Para dividir, podemos utilizar la regla de tres: 120 cm ÷ 15 cm = ?
4. Al dividir, obtenemos: 120 ÷ 15 = 8

¡Eso significa que podemos cortar 8 trozos de 15 cm de largo de la cinta!

Así que la respuesta es: 8 trozos de 15 cm de largo.


#### Chain-of-Thought  2

In [3]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

problema = """
Deseo recordar el nombre de un escritor argentino famoso por sus cuentos cortos. \
Sus obras fueron prohibidas por la última dictadura militar en Argentina. Se exilió a Francia. Ya falleció. \
Sus últimos años de vida transcurrieron en París, donde falleció."""

instrucciones = """Planteo del problema: 

Primero, haga una lista de los subproblemas que plantea este problema y que deben ser resueltos para llegar a la respuesta correcta.
Segundo, resuelva cada subproblema apoyándose en las soluciones previas.
Tercero, muestre la respuesta final.
Por favor siga las instrucciones para un buen resultado.
"""

template = """
Problema:
{problema}

Instrucciones:
{instrucciones}
"""

llm = ChatGroq(model="llama3-8b-8192",
               temperature=0.1,
               max_tokens=1000,
               verbose=True)   # Se instancia un objeto ChatGroq

prompt = PromptTemplate(template=template, input_variables=['problema', 'instrucciones'])
prompt_final = prompt.format(problema=problema, instrucciones=instrucciones)

respuesta = llm.invoke(prompt_final)
print(respuesta.content)

Excelente problema! Vamos a resolverlo paso a paso.

**Lista de subproblemas:**

1. ¿Quién es el escritor argentino famoso por sus cuentos cortos?
2. ¿Por qué sus obras fueron prohibidas por la última dictadura militar en Argentina?
3. ¿Dónde se exilió el escritor?
4. ¿Cuándo falleció el escritor?
5. ¿Dónde pasó sus últimos años de vida y dónde falleció?

**Resolución de los subproblemas:**

1. ¿Quién es el escritor argentino famoso por sus cuentos cortos?

No hay mucha información disponible sobre este escritor, pero podemos buscar en la literatura argentina y ver quién se ajusta a los demás subproblemas.

2. ¿Por qué sus obras fueron prohibidas por la última dictadura militar en Argentina?

La última dictadura militar en Argentina fue la dictadura autodenominada "Proceso de Reorganización Nacional", que gobernó el país desde 1976 hasta 1983. Es probable que las obras del escritor fueran prohibidas debido a su contenido político o social crítico.

3. ¿Dónde se exilió el escritor?

La 